# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [211]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re

import nltk

nltk.download("wordnet")
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download("punkt")
from nltk.tokenize import word_tokenize

nltk.download("stopwords")
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer



from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV

import pickle
import joblib

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\carst\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\carst\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carst\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [212]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('messages_categorized', engine)

categories = df.columns[4:]
Y = df[categories]
X = df['message']

# Replace 2 with 1 in related category
Y['related'][Y['related']==2] = 1


### 2. Write a tokenization function to process your text data


In [133]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [170]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

print("Set up pipeline with multi-output random forest")

Set up pipeline with multi-output random forest


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [204]:
import warnings
warnings.filterwarnings("ignore")

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)
pipeline.fit(X_train, y_train)

print("Model done fitting with pipeline.")



Model done fitting with pipeline.


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.



In [137]:
y_pred = pd.DataFrame(pipeline.predict(X_test))
y_pred.columns = categories
y_pred.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [209]:
print("Overall score: " + str(pipeline.score(X_test, y_test)))

for category in categories:
    print("Report for category: " + category + '\n')
    print(classification_report(y_test[category], y_pred[category]))
    print('\n')

Overall score: 0.2479755538579068
Report for category: related

              precision    recall  f1-score   support

           0       0.66      0.46      0.54      1568
           1       0.85      0.92      0.88      4977

    accuracy                           0.81      6545
   macro avg       0.75      0.69      0.71      6545
weighted avg       0.80      0.81      0.80      6545



Report for category: request

              precision    recall  f1-score   support

           0       0.89      0.97      0.93      5411
           1       0.78      0.44      0.56      1134

    accuracy                           0.88      6545
   macro avg       0.84      0.71      0.75      6545
weighted avg       0.87      0.88      0.87      6545



Report for category: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6513
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6545
  

### 6. Improve your model
Use grid search to find better parameters. 

In [184]:
parameters = {
   # 'vect__ngram_range': ((1,1),(1,2)),
#    'vect__max_df': (0.5, 0.75, 1.0),
 #   'vect__max_features': (None, 5000, 10000),
    'clf__estimator__n_estimators': [50, 100, 200]
   # 'clf__estimator__min_samples_split': [2, 3, 4]

}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, n_jobs=4)

print("Params for grid search defined.")

Params for grid search defined.


In [185]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   3 out of   9 | elapsed:  1.6min remaining:  3.2min
[Parallel(n_jobs=4)]: Done   4 out of   9 | elapsed:  3.0min remaining:  3.8min
[Parallel(n_jobs=4)]: Done   5 out of   9 | elapsed:  4.5min remaining:  3.6min
[Parallel(n_jobs=4)]: Done   6 out of   9 | elapsed:  4.6min remaining:  2.3min
[Parallel(n_jobs=4)]: Done   7 out of   9 | elapsed:  7.4min remaining:  2.1min
[Parallel(n_jobs=4)]: Done   9 out of   9 | elapsed: 10.4min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   9 out of   9 | elapsed: 10.4min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [189]:
y_pred_optimized = pd.DataFrame(cv.predict(X_test))
y_pred_optimized.columns = categories

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [208]:
print("Overall score: " + str(cv.score(X_test, y_test)))

for category in categories:
    print("Report for category: " + category + '\n')
    print(classification_report(y_test[category], y_pred_optimized[category]))
    print('\n')


Overall score: 0.2760886172650879
Report for category: related

              precision    recall  f1-score   support

           0       0.73      0.40      0.52      1568
           1       0.84      0.95      0.89      4977

    accuracy                           0.82      6545
   macro avg       0.78      0.68      0.71      6545
weighted avg       0.81      0.82      0.80      6545



Report for category: request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5411
           1       0.82      0.48      0.61      1134

    accuracy                           0.89      6545
   macro avg       0.86      0.73      0.77      6545
weighted avg       0.89      0.89      0.88      6545



Report for category: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6513
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6545
  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF


### 9. Export your model as a pickle file

In [199]:
filename = 'disaster_response_model.sav'
pickle.dump(cv, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, y_test)
print(result)


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.


